In [2]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import joblib
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [3]:

# Cargar el dataset

#metadata_path = '/Users/luiseduardogarciablanco/Desktop/nueva data cancer/prueba_data_16_20/dataset_16_20_emb.csv'
metadata = pd.read_csv('/Users/luiseduardogarciablanco/Desktop/nueva data cancer/metadata model/df_train_processed.csv')  


# Ruta a las imágenes
base_image_path = "/Users/luiseduardogarciablanco/Desktop/nueva data cancer/test web/jpg_ensambled_model"


# Mostrar información básica del dataset
print (metadata.head())
print(metadata['target'].value_counts())

/var/folders/sc/w6cv9_291mqc4c1m0qn06_4m0000gn/T/ipykernel_52712/1891167088.py:4: DtypeWarning: Columns (49,50,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('/Users/luiseduardogarciablanco/Desktop/nueva data cancer/metadata model/df_train_processed.csv')


   Unnamed: 0       isic_id  target  patient_id  age_approx   sex  \
0           0  ISIC_0015670       0  IP_1235828        60.0  male   
1           1  ISIC_0015845       0  IP_8170065        60.0  male   
2           2  ISIC_0015864       0  IP_6724798        60.0  male   
3           3  ISIC_0015902       0  IP_4111386        65.0  male   
4           4  ISIC_0024200       0  IP_8313778        55.0  male   

  anatom_site_general  clin_size_long_diam_mm          image_type  \
0     lower extremity                    3.04  TBP tile: close-up   
1           head/neck                    1.10  TBP tile: close-up   
2     posterior torso                    3.40  TBP tile: close-up   
3      anterior torso                    3.22  TBP tile: close-up   
4      anterior torso                    2.73  TBP tile: close-up   

  tbp_tile_type  ...  onehot_33  onehot_34  onehot_35  onehot_36  onehot_37  \
0     3D: white  ...          1          0          0          0          0   
1     3D: wh

In [4]:


# Cargar el dataset
df = metadata

# Dividir el DataFrame en dos partes según el valor del target
df_majority = df[df['target'] == 0]
df_minority = df[df['target'] == 1]

# Reducir el número de muestras de la clase mayoritaria a 6766 para quilibrar las muestras
df_majority_downsampled = df_majority.sample(n=500, random_state=42)

# Combinar los DataFrames balanceados
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Opcional: Mezclar el DataFrame balanceado para asegurar aleatoriedad
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Mostrar la distribución de clases después del balanceo
print(df_balanced['target'].value_counts())

df_balanced.info()

target
0    500
1    389
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Columns: 295 entries, Unnamed: 0 to onehot_42
dtypes: float64(229), int64(47), object(19)
memory usage: 2.0+ MB


In [5]:
df=df_balanced
df

,Unnamed: 0,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,...,onehot_33,onehot_34,onehot_35,onehot_36,onehot_37,onehot_38,onehot_39,onehot_40,onehot_41,onehot_42
0,196309,ISIC_5046058,0,IP_1507554,30.0,male,upper extremity,2.82,TBP tile: close-up,3D: XP,...,0,0,0,0,1,0,0,0,0,0
1,59691,ISIC_1577832,0,IP_8607082,65.0,male,posterior torso,2.51,TBP tile: close-up,3D: white,...,0,1,0,0,0,0,0,1,0,0
2,266496,ISIC_6818565,0,IP_8008363,50.0,female,anterior torso,2.82,TBP tile: close-up,3D: XP,...,0,0,1,0,1,0,0,0,0,0
3,256165,ISIC_6565046,0,IP_5568236,45.0,female,upper extremity,2.70,TBP tile: close-up,3D: XP,...,0,0,0,0,0,0,0,0,1,0
4,79811,ISIC_2090145,1,IP_8078890,80.0,male,head/neck,4.68,TBP tile: close-up,3D: white,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,234432,ISIC_6020105,0,IP_1421772,55.0,male,upper extremity,2.60,TBP tile: close-up,3D: XP,...,0,0,0,0,0,0,0,0,1,0
885,326865,ISIC_8349201,0,IP_9409053,75.0,female,upper extremity,4.26,TBP tile: close-up,3D: XP,...,0,0,0,0,1,0,0,0,0,0
886,351539,ISIC_8966407,1,IP_2732171,65.0,male,head/neck,4.18,TBP tile: close-up,3D: white,...,0,0,0,0,0,0,0,1,0,0
887,188867,ISIC_4856214,0,IP_6934330,60.0,male,posterior torso,3.70,TBP tile: close-up,3D: XP,...,0,1,0,0,0,0,0,0,1,0


empezamos con el ensamblado de los dos modelos entrenados, tanto el de imagenes como el de metadata, con el fin de entrenar un tercer modelo con el resultado de la prediccion de ambos modelos vs el target 

In [6]:
# Cargar el modelo de imágenes (ResNet50)
image_model = load_model('/Users/luiseduardogarciablanco/Desktop/bootcamp/Proyecto_Final_Cancer/src/ensamblado_de_modelos/model/resnet50_fine_tuned_model_20.h5')

# Cargar el modelo de metadatos
with open('/Users/luiseduardogarciablanco/Desktop/bootcamp/Proyecto_Final_Cancer/src/ensamblado_de_modelos/model/metadata_model.pkl', 'rb') as f:
    metadata_model = joblib.load(f)

2024-09-04 18:15:19.759802: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-09-04 18:15:19.759844: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-09-04 18:15:19.759854: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-09-04 18:15:19.760283: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-04 18:15:19.760345: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# Ver el número de características
print(f"Número de características del modelo: {metadata_model.n_features_in_}")

Número de características del modelo: 272


al comprobar que el numero de caractristicas es diferente al numero de variables que tenemos en el dataframe vamos a revisar que columnas difieren en los modelos

In [8]:
# primero verificaremos los modelos base dentro del VotingClassifier
for name, pipeline in metadata_model.named_estimators_.items():
    if hasattr(pipeline, 'named_steps'):
        print(f"Pipeline: {name}")
        for step_name, step in pipeline.named_steps.items():
            print(f"  Paso: {step_name} - {step}")
    else:
        print(f"Modelo base: {name} - No es un pipeline")

Pipeline: lgb
  Paso: sampler_1 - RandomOverSampler(random_state=42, sampling_strategy=0.003)
  Paso: sampler_2 - RandomUnderSampler(random_state=42, sampling_strategy=0.01)
  Paso: classifier - LGBMClassifier(bagging_fraction=0.5630352613173868, bagging_freq=5,
               colsample_bynode=0.6558724487912035,
               colsample_bytree=0.6657330406644436,
               lambda_l1=0.014902669085932872, lambda_l2=0.03957395263238096,
               learning_rate=0.014993253193227397, max_depth=7,
               min_data_in_leaf=20, num_leaves=34, objective='binary',
               scale_pos_weight=2.4365960001172486, verbosity=-1)
Pipeline: cb
  Paso: sampler_1 - RandomOverSampler(random_state=42, sampling_strategy=0.003)
  Paso: sampler_2 - RandomUnderSampler(random_state=42, sampling_strategy=0.01)
  Paso: classifier - <catboost.core.CatBoostClassifier object at 0x35a22a430>
Pipeline: xgb
  Paso: sampler_1 - RandomOverSampler(random_state=42, sampling_strategy=0.003)
  Paso: s

In [9]:
# Obtener los nombres de las características para el LightGBM, ya que los otros 2 modelos trabajan con la misma cantidad de variables
for name, pipeline in metadata_model.named_estimators_.items():
    if hasattr(pipeline.named_steps['classifier'], 'feature_name_'):
        feature_names_lgb = pipeline.named_steps['classifier'].feature_name_
        print(f"Características para LightGBM: {feature_names_lgb}")
    else:
        print(f"LightGBM no tiene atributo 'feature_name_'")

Características para LightGBM: ['age_approx', 'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color', 'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z', 'lesion_size_ratio', 'lesion_shape_index', 'hue_contrast', 'luminance_contrast', 'lesion_color_difference', 'border_complexity', 'color_uniformity', 'position_distance_3d', 'perimeter_to_area_ratio', 'area_to_perimeter_ratio', 'lesion_visibility_score', 'symmetry_border_consistency', 'consistency_symmetry_border', 'color_consistency', 'consistenc

In [10]:
#enumeramos la lista de variables con las que trabajan los modelos entrenados para el dataset
feature_names_lgb = ['age_approx', 'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 
                     'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 
                     'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm', 
                     'tbp_lv_eccentricity', 'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border', 'tbp_lv_norm_color', 
                     'tbp_lv_perimeterMM', 'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 
                     'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z', 'lesion_size_ratio', 'lesion_shape_index', 'hue_contrast', 
                     'luminance_contrast', 'lesion_color_difference', 'border_complexity', 'color_uniformity', 'position_distance_3d', 
                     'perimeter_to_area_ratio', 'area_to_perimeter_ratio', 'lesion_visibility_score', 'symmetry_border_consistency', 
                     'consistency_symmetry_border', 'color_consistency', 'consistency_color', 'size_age_interaction', 'hue_color_std_interaction', 
                     'lesion_severity_index', 'shape_complexity_index', 'color_contrast_index', 'log_lesion_area', 'normalized_lesion_size', 
                     'mean_hue_difference', 'std_dev_contrast', 'color_shape_composite_index', 'lesion_orientation_3d', 'overall_color_difference', 
                     'symmetry_perimeter_interaction', 'comprehensive_lesion_index', 'color_variance_ratio', 'border_color_interaction', 
                     'border_color_interaction_2', 'size_color_contrast_ratio', 'age_normalized_nevi_confidence', 'age_normalized_nevi_confidence_2', 
                     'color_asymmetry_index', 'volume_approximation_3d', 'color_range', 'shape_color_consistency', 'border_length_ratio', 
                     'age_size_symmetry_index', 'index_age_size_symmetry', 'age_approx_patient_norm', 'clin_size_long_diam_mm_patient_norm', 
                     'tbp_lv_A_patient_norm', 'tbp_lv_Aext_patient_norm', 'tbp_lv_B_patient_norm', 'tbp_lv_Bext_patient_norm', 'tbp_lv_C_patient_norm', 
                     'tbp_lv_Cext_patient_norm', 'tbp_lv_H_patient_norm', 'tbp_lv_Hext_patient_norm', 'tbp_lv_L_patient_norm', 'tbp_lv_Lext_patient_norm',
                     'tbp_lv_areaMM2_patient_norm', 'tbp_lv_area_perim_ratio_patient_norm', 'tbp_lv_color_std_mean_patient_norm', 
                     'tbp_lv_deltaA_patient_norm', 'tbp_lv_deltaB_patient_norm', 'tbp_lv_deltaL_patient_norm', 'tbp_lv_deltaLB_patient_norm', 
                     'tbp_lv_deltaLBnorm_patient_norm', 'tbp_lv_eccentricity_patient_norm', 'tbp_lv_minorAxisMM_patient_norm', 
                     'tbp_lv_nevi_confidence_patient_norm', 'tbp_lv_norm_border_patient_norm', 'tbp_lv_norm_color_patient_norm', 
                     'tbp_lv_perimeterMM_patient_norm', 'tbp_lv_radial_color_std_max_patient_norm', 'tbp_lv_stdL_patient_norm',
                     'tbp_lv_stdLExt_patient_norm', 'tbp_lv_symm_2axis_patient_norm', 'tbp_lv_symm_2axis_angle_patient_norm', 'tbp_lv_x_patient_norm', 
                     'tbp_lv_y_patient_norm', 'tbp_lv_z_patient_norm', 'lesion_size_ratio_patient_norm', 'lesion_shape_index_patient_norm', 
                     'hue_contrast_patient_norm', 'luminance_contrast_patient_norm', 'lesion_color_difference_patient_norm', 'border_complexity_patient_norm',
                     'color_uniformity_patient_norm', 'position_distance_3d_patient_norm', 'perimeter_to_area_ratio_patient_norm', 
                     'area_to_perimeter_ratio_patient_norm', 'lesion_visibility_score_patient_norm', 'symmetry_border_consistency_patient_norm', 
                     'consistency_symmetry_border_patient_norm', 'color_consistency_patient_norm', 'consistency_color_patient_norm', 
                     'size_age_interaction_patient_norm', 'hue_color_std_interaction_patient_norm', 'lesion_severity_index_patient_norm',
                     'shape_complexity_index_patient_norm', 'color_contrast_index_patient_norm', 'log_lesion_area_patient_norm', 
                     'normalized_lesion_size_patient_norm', 'mean_hue_difference_patient_norm', 'std_dev_contrast_patient_norm', 
                     'color_shape_composite_index_patient_norm', 'lesion_orientation_3d_patient_norm', 'overall_color_difference_patient_norm', 
                     'symmetry_perimeter_interaction_patient_norm', 'comprehensive_lesion_index_patient_norm', 'color_variance_ratio_patient_norm', 
                     'border_color_interaction_patient_norm', 'border_color_interaction_2_patient_norm', 'size_color_contrast_ratio_patient_norm', 
                     'age_normalized_nevi_confidence_patient_norm', 'age_normalized_nevi_confidence_2_patient_norm', 'color_asymmetry_index_patient_norm', 
                     'volume_approximation_3d_patient_norm', 'color_range_patient_norm', 'shape_color_consistency_patient_norm', 'border_length_ratio_patient_norm', 
                     'age_size_symmetry_index_patient_norm', 'index_age_size_symmetry_patient_norm', 'age_approx_sex_norm', 'clin_size_long_diam_mm_sex_norm', 
                     'tbp_lv_A_sex_norm', 'tbp_lv_Aext_sex_norm', 'tbp_lv_B_sex_norm', 'tbp_lv_Bext_sex_norm', 'tbp_lv_C_sex_norm', 'tbp_lv_Cext_sex_norm', 
                     'tbp_lv_H_sex_norm', 'tbp_lv_Hext_sex_norm', 'tbp_lv_L_sex_norm', 'tbp_lv_Lext_sex_norm', 'tbp_lv_areaMM2_sex_norm', 
                     'tbp_lv_area_perim_ratio_sex_norm', 'tbp_lv_color_std_mean_sex_norm', 'tbp_lv_deltaA_sex_norm', 'tbp_lv_deltaB_sex_norm', 
                     'tbp_lv_deltaL_sex_norm', 'tbp_lv_deltaLB_sex_norm', 'tbp_lv_deltaLBnorm_sex_norm', 'tbp_lv_eccentricity_sex_norm', 
                     'tbp_lv_minorAxisMM_sex_norm', 'tbp_lv_nevi_confidence_sex_norm', 'tbp_lv_norm_border_sex_norm', 'tbp_lv_norm_color_sex_norm', 
                     'tbp_lv_perimeterMM_sex_norm', 'tbp_lv_radial_color_std_max_sex_norm', 'tbp_lv_stdL_sex_norm', 'tbp_lv_stdLExt_sex_norm', 
                     'tbp_lv_symm_2axis_sex_norm', 'tbp_lv_symm_2axis_angle_sex_norm', 'tbp_lv_x_sex_norm', 'tbp_lv_y_sex_norm', 'tbp_lv_z_sex_norm', 
                     'lesion_size_ratio_sex_norm', 'lesion_shape_index_sex_norm', 'hue_contrast_sex_norm', 'luminance_contrast_sex_norm', 
                     'lesion_color_difference_sex_norm', 'border_complexity_sex_norm', 'color_uniformity_sex_norm', 'position_distance_3d_sex_norm', 
                     'perimeter_to_area_ratio_sex_norm', 'area_to_perimeter_ratio_sex_norm', 'lesion_visibility_score_sex_norm', 
                     'symmetry_border_consistency_sex_norm', 'consistency_symmetry_border_sex_norm', 'color_consistency_sex_norm', 
                     'consistency_color_sex_norm', 'size_age_interaction_sex_norm', 'hue_color_std_interaction_sex_norm', 'lesion_severity_index_sex_norm', 
                     'shape_complexity_index_sex_norm', 'color_contrast_index_sex_norm', 'log_lesion_area_sex_norm', 'normalized_lesion_size_sex_norm', 
                     'mean_hue_difference_sex_norm', 'std_dev_contrast_sex_norm', 'color_shape_composite_index_sex_norm', 'lesion_orientation_3d_sex_norm', 
                     'overall_color_difference_sex_norm', 'symmetry_perimeter_interaction_sex_norm', 'comprehensive_lesion_index_sex_norm', 
                     'color_variance_ratio_sex_norm', 'border_color_interaction_sex_norm', 'border_color_interaction_2_sex_norm', 'size_color_contrast_ratio_sex_norm', 
                     'age_normalized_nevi_confidence_sex_norm', 'age_normalized_nevi_confidence_2_sex_norm', 'color_asymmetry_index_sex_norm', 'volume_approximation_3d_sex_norm', 
                     'color_range_sex_norm', 'shape_color_consistency_sex_norm', 'border_length_ratio_sex_norm', 'age_size_symmetry_index_sex_norm', 'index_age_size_symmetry_sex_norm', 
                     'count_per_patient', 'onehot_0', 'onehot_1', 'onehot_2', 'onehot_3', 'onehot_4', 'onehot_5', 'onehot_6', 'onehot_7', 'onehot_8', 'onehot_9', 'onehot_10', 
                     'onehot_11', 'onehot_12', 'onehot_13', 'onehot_14', 'onehot_15', 'onehot_16', 'onehot_17', 'onehot_18', 'onehot_19', 'onehot_20', 'onehot_21', 
                     'onehot_22', 'onehot_23', 'onehot_24', 'onehot_25', 'onehot_26', 'onehot_27', 'onehot_28', 'onehot_29', 'onehot_30', 'onehot_31', 'onehot_32', 
                     'onehot_33', 'onehot_34', 'onehot_35', 'onehot_36', 'onehot_37', 'onehot_38', 'onehot_39', 'onehot_40', 'onehot_41', 'onehot_42']

num_features_lgb = len(feature_names_lgb)
print(f"Número de características en feature_names_lgb: {num_features_lgb}")

Número de características en feature_names_lgb: 272


hacemos una comprobacion entre las variables con las que trabaja el modelo y las variables de nuestro dataframe para verificar las diferencias

In [11]:
# Obtener la lista de columnas del DataFrame
df_columns = df.columns.tolist()

# Contar el número de columnas en el DataFrame
num_columns_df = len(df_columns)
print(f"Número de columnas en el DataFrame: {num_columns_df}")

# Comparar la lista de características con las columnas del DataFrame
missing_features = [feature for feature in feature_names_lgb if feature not in df_columns]
extra_columns = [column for column in df_columns if column not in feature_names_lgb]

# Imprimir las diferencias
print(f"Características en feature_names_lgb pero no en el DataFrame: {len(missing_features)}")
print("Características faltantes:")
print(missing_features)

print(f"Columnas en el DataFrame pero no en feature_names_lgb: {len(extra_columns)}")
print("Columnas adicionales en el DataFrame:")
print(extra_columns)

Número de columnas en el DataFrame: 295
Características en feature_names_lgb pero no en el DataFrame: 0
Características faltantes:
[]
Columnas en el DataFrame pero no en feature_names_lgb: 23
Columnas adicionales en el DataFrame:
['Unnamed: 0', 'isic_id', 'target', 'patient_id', 'sex', 'anatom_site_general', 'image_type', 'tbp_tile_type', 'tbp_lv_location', 'tbp_lv_location_simple', 'attribution', 'copyright_license', 'lesion_id', 'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5', 'mel_mitotic_index', 'mel_thick_mm', 'tbp_lv_dnn_lesion_confidence', 'combined_anatomical_site']


ya que tenmos las columnas sobrantes de nuestro dataframe, procedremos a eliminar las columnas con las que no fueron entrenados los modelos

In [12]:
# Lista de columnas a eliminar
columns_to_drop = ['Unnamed: 0', 'patient_id', 'sex', 'anatom_site_general', 'image_type', 'tbp_tile_type', 
                   'tbp_lv_location', 'tbp_lv_location_simple', 'attribution', 'copyright_license', 
                   'lesion_id', 'iddx_full', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4', 'iddx_5', 
                   'mel_mitotic_index', 'mel_thick_mm', 'tbp_lv_dnn_lesion_confidence', 
                   'combined_anatomical_site']

# Eliminar las columnas del DataFrame
df = df.drop(columns=columns_to_drop)

# Sumar los valores de todas las columnas restantes
column_sums = df.sum()

# Mostrar la suma de cada columna
print("Suma de los valores en cada columna después de eliminar las especificadas:")
print(column_sums)

Suma de los valores en cada columna después de eliminar las especificadas:
isic_id                   ISIC_5046058ISIC_1577832ISIC_6818565ISIC_65650...
target                                                                  389
age_approx                                                          52805.0
clin_size_long_diam_mm                                              4177.82
tbp_lv_A                                                       18765.821823
                                                ...                        
onehot_38                                                                21
onehot_39                                                               142
onehot_40                                                               327
onehot_41                                                               104
onehot_42                                                                24
Length: 274, dtype: object


ya tenemos las 274 columnas, que luego de eliminar las variables identificativas y del target, quedaran solo las columnas con las que trabajan los modelos

Cargamos el modelo de imagenes y hacemos las pruebas de funcionamiento

In [24]:
import numpy as np
from PIL import Image
from tensorflow.keras.applications.resnet50 import preprocess_input

# Definir la función de preprocesamiento
def preprocess_image(image_path, size=(256, 256)):
    with Image.open(image_path) as img:
        img = img.resize(size)
        img_array = np.array(img)
        img_array = preprocess_input(img_array)  # Normalización específica de ResNet50
        img_array = np.expand_dims(img_array, axis=0)  # Añadir dimensión para el batch
        return img_array

# Probar la función de preprocesamiento con una imagen de ejemplo
example_image_path = '/Users/luiseduardogarciablanco/Desktop/nueva data cancer/test web/jpg_ensambled_model/ISIC_0527743.jpg'
preprocessed_image = preprocess_image(example_image_path)
print("Forma de la imagen preprocesada:", preprocessed_image.shape)

Forma de la imagen preprocesada: (1, 256, 256, 3)


In [25]:
from tensorflow.keras.models import load_model

# Cargar el modelo de imágenes
image_model_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Proyecto_Final_Cancer/src/ensamblado_de_modelos/model/resnet50_fine_tuned_model_20.h5'
image_model = load_model(image_model_path)

# Definir la función de predicción
def predict_image(image_path, image_model):
    img = preprocess_image(image_path)
    img_pred = image_model.predict(img)
    return img_pred.flatten()  # Aplanar para que tenga la forma correcta

# Probar la función de predicción con la imagen preprocesada
image_prediction = predict_image(example_image_path, image_model)
print("Predicción de la imagen:", image_prediction)

2024-09-04 18:39:11.250058: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 792ms/step
Predicción de la imagen: [0.46279442]


In [26]:
import joblib

# Cargar el modelo de metadatos
metadata_model_path = '/Users/luiseduardogarciablanco/Desktop/bootcamp/Proyecto_Final_Cancer/src/ensamblado_de_modelos/model/metadata_model.pkl'
metadata_model = joblib.load(metadata_model_path)

# Función para extraer el image_id del path
def extract_image_id_from_path(base_image_path):
    return os.path.splitext(os.path.basename(base_image_path))[0]

# Predicción del modelo de metadatos
def predict_metadata(df, metadata_model, image_id):
    # Obtener los metadatos correspondientes al image_id
    metadata = df[df['isic_id'] == image_id].drop(columns=['isic_id', 'target'])
    
    # Predicción del modelo de metadatos
    pred = metadata_model.predict_proba(metadata)[:, 1]  # Probabilidad de la clase positiva
    
    return pred

# Probar la función de predicción de metadatos
example_image_id = extract_image_id_from_path(example_image_path)

# Eliminar el prefijo "0_" o "1_" si está presente
if example_image_id.startswith("0_") or example_image_id.startswith("1_"):
    example_image_id = example_image_id[2:]

metadata_prediction = predict_metadata(df, metadata_model, example_image_id)
print("Predicción de metadatos:", metadata_prediction)

Predicción de metadatos: [0.5982233]


Crear el Modelo Ensamblador

Para combinar las predicciones, puedes usar un modelo simple como LogisticRegression, RandomForestClassifier, o cualquier otro modelo que prefieras para la etapa de ensamblado.

Realizar Predicciones con el Modelo Ensamblador

Finalmente, usa el modelo ensamblador para hacer predicciones en nuevos datos.

In [29]:
# Función para entrenar el modelo ensamblador
def train_ensemble_model(image_paths, df, image_model, metadata_model):
    X = []
    y = []
    
    for image_path in image_paths:
        image_id = extract_image_id_from_path(image_path)
        
        # Obtener la predicción de la imagen (suponiendo que retorna un array 1D)
        image_pred = predict_image(image_path, image_model)
        
        # Obtener la predicción de los metadatos (suponiendo que retorna un valor escalar o 1D)
        metadata_pred = predict_metadata(df, metadata_model, image_id)
        
        # Asegurarse de que metadata_pred sea un array 1D
        if metadata_pred.ndim == 2 and metadata_pred.shape[1] == 1:
            metadata_pred = metadata_pred.flatten()
        
        # Combinar las predicciones
        features = np.concatenate([image_pred, metadata_pred])
        X.append(features)
        
        # Obtener el target (suponiendo que está en el DataFrame)
        target = df[df['isic_id'] == image_id]['target'].values[0]
        y.append(target)
    
    X = np.array(X)
    y = np.array(y)
    
    # Dividir en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Crear y entrenar el modelo ensamblador
    ensemble_model = LogisticRegression()
    ensemble_model.fit(X_train, y_train)
    
    return ensemble_model, X_test, y_test


In [37]:
# Entrenar el modelo ensamblador
# Obtener todas las rutas completas de las imágenes desde el DataFrame
image_paths = [os.path.join(base_image_path, f"{image_id}.jpg") for image_id in df['isic_id']]

# Entrenar el modelo ensamblador
ensemble_model, X_test, y_test = train_ensemble_model(image_paths, df, image_model, metadata_model)


1/1 [==============================] - 0s 24ms/step


In [38]:
# --------------------------------------------
# Generar informes del modelo ensamblador
# --------------------------------------------
from sklearn.metrics import classification_report, confusion_matrix

def generate_reports(X_test, y_test, ensemble_model):
    y_pred = ensemble_model.predict(X_test)
    
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

# Generar informes
generate_reports(X_test, y_test, ensemble_model)

Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       104
           1       1.00      0.95      0.97        74

    accuracy                           0.98       178
   macro avg       0.98      0.97      0.98       178
weighted avg       0.98      0.98      0.98       178

Confusion Matrix:
[[104   0]
 [  4  70]]


In [39]:
#guardamos el modelo ensamblado
import joblib

# Ruta donde se guardará el modelo
model_path = "/Users/luiseduardogarciablanco/Desktop/bootcamp/Proyecto_Final_Cancer/src/ensamblado_de_modelos/model/ensemble_model.pkl"

# Guardar el modelo
joblib.dump(ensemble_model, model_path)

print(f"Modelo guardado en {model_path}")

Modelo guardado en /Users/luiseduardogarciablanco/Desktop/bootcamp/Proyecto_Final_Cancer/src/ensamblado_de_modelos/model/ensemble_model.pkl


In [42]:
import pandas as pd

# Ruta al archivo CSV
csv_path = "/Users/luiseduardogarciablanco/Desktop/bootcamp/Flask_proyecto_cancer/static/data/dataset_flask.csv"

# Guardar el DataFrame en formato CSV
df.to_csv(csv_path, index=False)